In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [2]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [12]:
X_train = pd.read_csv('./X_train.csv')
y_train = pd.read_csv('./y_train.csv')
X_test = pd.read_csv('./X_test.csv')
y_test = pd.read_csv('./y_test.csv')

In [13]:
%store -r class_weight
%store -r X_train_SMOTE
%store -r y_train_SMOTE

In [28]:
X_train = pd.DataFrame(X_train.iloc[:, :-1], columns=X_train.columns[:-1])
X_test = pd.DataFrame(X_test.iloc[:, :-1], columns=X_test.columns[:-1])
#y_train = y_train['Tip']
#y_test = y_test['Tip']
y_train_SMOTE = pd.DataFrame(y_train_SMOTE, columns=['Tip'])
X_train_SMOTE = pd.DataFrame(X_train_SMOTE.iloc[:, :-1], columns=X_train_SMOTE.columns[:-1])

In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
oe = OrdinalEncoder()

In [17]:
oe.fit(y_train)

OrdinalEncoder()

In [18]:
y_train = pd.DataFrame(oe.transform(y_train), columns=y_train.columns)['Tip']

In [19]:
y_test = pd.DataFrame(oe.transform(y_test), columns=y_test.columns)['Tip']

In [31]:
oe.fit(y_train_SMOTE)

OrdinalEncoder()

In [32]:
y_train_SMOTE = pd.DataFrame(oe.transform(y_train_SMOTE), columns=y_train_SMOTE.columns)['Tip']

In [36]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)
classes_weights

array([0.3591716 , 1.10363636, 0.3591716 , 1.10363636, 0.3591716 ,
       1.61866667, 0.3591716 , 1.73428571, 0.3591716 , 8.67142857,
       0.3591716 , 0.3591716 , 1.73428571, 0.3591716 , 1.61866667,
       1.10363636, 1.10363636, 0.3591716 , 0.3591716 , 0.3591716 ,
       1.10363636, 0.3591716 , 1.61866667, 8.67142857, 1.10363636,
       1.10363636, 1.10363636, 0.3591716 , 0.3591716 , 0.3591716 ,
       1.10363636, 0.3591716 , 0.3591716 , 0.3591716 , 8.67142857,
       1.10363636, 0.3591716 , 0.3591716 , 0.3591716 , 1.61866667,
       1.73428571, 0.3591716 , 0.3591716 , 0.3591716 , 1.73428571,
       0.3591716 , 0.3591716 , 0.3591716 , 0.3591716 , 1.61866667,
       0.3591716 , 1.10363636, 0.3591716 , 1.10363636, 0.3591716 ,
       1.61866667, 0.3591716 , 0.3591716 , 0.3591716 , 0.3591716 ,
       0.3591716 , 1.61866667, 1.73428571, 1.10363636, 0.3591716 ,
       0.3591716 , 0.3591716 , 0.3591716 , 8.67142857, 1.10363636,
       0.3591716 , 0.3591716 , 1.61866667, 8.67142857, 0.35917

In [20]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2],
    'max_leaves': [0, 5],
    'booster': ['gbtree', 'gblinear'],
}

clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5)

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', verbose=4)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5, **best_params)
best_clf.fit(X_train, y_train)

y_pred_best = best_clf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

classification_rep = classification_report(y_test, y_pred_best)

print("\nNajbolji parametri:", best_params)
print("Preciznost modela sa najboljim parametrima:", round(accuracy_best, 5))
print("\nIzvestaj o klasifikaciji:\n", classification_rep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.795 total time=   2.4s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.803 total time=   0.8s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.884 total time=   0.8s
[CV 4/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.843 total time=   0.8s
[CV 5/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.843 total time=   0.8s
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.795 total time=   1.1s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.787 total time=   1.2s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.876 total time=   1.2s
[CV 4/5] END booster=gbtree, learning_rate

C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.623 total time=   0.0s
[CV 3/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.653 total time=   0.0s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 4/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.661 total time=   0.0s
[CV 5/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.636 total time=   0.0s
[CV 1/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.574 total time=   0.1s
[CV 2/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.623 total time=   0.1s


...

[CV 3/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.785 total time=   0.1s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 4/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.777 total time=   0.1s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:05:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 5/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.752 total time=   0.1s

Najbolji parametri: {'booster': 'gbtree', 'learning_rate': 0.2, 'max_leaves': 5, 'n_estimators': 300}
Preciznost modela sa najboljim parametrima: 0.81579

Izvestaj o klasifikaciji:
               precision    recall  f1-score   support

         0.0       0.91      0.94      0.92        82
         1.0       0.70      0.64      0.67        25
         2.0       0.88      0.88      0.88        25
         3.0       0.33      0.25      0.29         4
         4.0       0.50      0.50      0.50        16

    accuracy                           0.82       152
   macro avg       0.66      0.64      0.65       152
weighted avg       0.81      0.82      0.81       152



In [33]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2],
    'max_leaves': [0, 5],
    'booster': ['gbtree', 'gblinear'],
}

clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5)

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', verbose=4)
grid_search.fit(X_train_SMOTE, y_train_SMOTE)

best_params = grid_search.best_params_

best_clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5, **best_params)
best_clf.fit(X_train_SMOTE, y_train_SMOTE)

y_pred_best = best_clf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

classification_rep = classification_report(y_test, y_pred_best)

print("\nNajbolji parametri:", best_params)
print("Preciznost modela sa najboljim parametrima:", round(accuracy_best, 5))
print("\nIzvestaj o klasifikaciji:\n", classification_rep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.896 total time=   1.1s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.944 total time=   1.1s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.950 total time=   1.3s
[CV 4/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.956 total time=   1.2s
[CV 5/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.941 total time=   1.2s
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.914 total time=   1.8s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.941 total time=   1.9s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.959 total time=   1.9s
[CV 4/5] END booster=gbtree, learning_rate

C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.568 total time=   0.1s
[CV 3/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.562 total time=   0.1s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 4/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.577 total time=   0.1s
[CV 5/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.530 total time=   0.1s


...

[CV 3/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.654 total time=   0.3s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 4/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.624 total time=   0.3s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:12:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 5/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.630 total time=   0.3s

Najbolji parametri: {'booster': 'gbtree', 'learning_rate': 0.1, 'max_leaves': 0, 'n_estimators': 300}
Preciznost modela sa najboljim parametrima: 0.81579

Izvestaj o klasifikaciji:
               precision    recall  f1-score   support

         0.0       0.92      0.95      0.93        82
         1.0       0.68      0.60      0.64        25
         2.0       0.96      0.88      0.92        25
         3.0       0.20      0.25      0.22         4
         4.0       0.47      0.50      0.48        16

    accuracy                           0.82       152
   macro avg       0.65      0.64      0.64       152
weighted avg       0.82      0.82      0.82       152



In [38]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2],
    'max_leaves': [0, 5],
    'booster': ['gbtree', 'gblinear'],
}

clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5)

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', verbose=4)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_clf = XGBClassifier(objective="multi:softmax", random_state=42, num_class=5, **best_params, sample_weight=classes_weights)
best_clf.fit(X_train, y_train)

y_pred_best = best_clf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

classification_rep = classification_report(y_test, y_pred_best)

print("\nNajbolji parametri:", best_params)
print("Preciznost modela sa najboljim parametrima:", round(accuracy_best, 5))
print("\nIzvestaj o klasifikaciji:\n", classification_rep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.803 total time=   0.9s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.762 total time=   0.8s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.860 total time=   0.8s
[CV 4/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.835 total time=   0.8s
[CV 5/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.860 total time=   0.8s
[CV 1/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.803 total time=   1.2s
[CV 2/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.770 total time=   1.2s
[CV 3/5] END booster=gbtree, learning_rate=0.1, max_leaves=0, n_estimators=200;, score=0.876 total time=   1.3s
[CV 4/5] END booster=gbtree, learning_rate

C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.615 total time=   0.0s
[CV 3/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.653 total time=   0.0s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 4/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.661 total time=   0.0s
[CV 5/5] END booster=gblinear, learning_rate=0.1, max_leaves=0, n_estimators=100;, score=0.636 total time=   0.0s


...

[CV 4/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.752 total time=   0.2s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 5/5] END booster=gblinear, learning_rate=0.2, max_leaves=5, n_estimators=300;, score=0.752 total time=   0.2s


C:\Users\mihai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:26:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sample_weight" } are not used.

  warnings.warn(smsg, UserWarning)



Najbolji parametri: {'booster': 'gbtree', 'learning_rate': 0.2, 'max_leaves': 5, 'n_estimators': 200}
Preciznost modela sa najboljim parametrima: 0.81579

Izvestaj o klasifikaciji:
               precision    recall  f1-score   support

         0.0       0.91      0.94      0.92        82
         1.0       0.70      0.64      0.67        25
         2.0       0.88      0.88      0.88        25
         3.0       0.33      0.25      0.29         4
         4.0       0.50      0.50      0.50        16

    accuracy                           0.82       152
   macro avg       0.66      0.64      0.65       152
weighted avg       0.81      0.82      0.81       152

